In [7]:
#invoke libraries
import codecs
import nltk
from nltk import word_tokenize, pos_tag
from sklearn.model_selection import train_test_split
import pycrfsuite
import os, os.path, sys
import glob
from xml.etree import ElementTree
import numpy as np
from sklearn.metrics import classification_report

In [2]:
from os import listdir
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

import numpy as np

from sklearn.datasets import make_classification

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix



output_path_name = "task9.2_raquel_60.txt"

output_path = "evaluations/" + output_path_name
results_path = output_path.replace('.txt', '_All_scores.log')
datadir = '../../data/Test-DDI/DrugBank'
training_data = '/home/raquel/Documents/mai/ahlt/data/Train/All'
train_df_path = '/home/raquel/Documents/mai/ahlt/data/DF/train.csv'

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df = pd.read_csv(train_df_path, index_col=0)

sentences = train_df.sentence_text.values
y = train_df['relation_type'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000,stratify=y)


def vectorize_data(sentences_train,sentences_test):  
    
    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    
    X_train = vectorizer.transform(sentences_train)
    X_test =  vectorizer.transform(sentences_test)
    
    return X_train, X_test

X_train, X_test = vectorize_data(sentences_train,sentences_test)

In [12]:
def tokenize(txt):
    offset = 0
    tks = []
    # word_tokenize splits words, taking into account punctuations, numbers, etc.
    for t in word_tokenize(txt):
        # keep track of the position where each token should appear, and
        # store that information with the token
        offset = txt.find(t, offset)
        tks.append((t, offset, offset + len(t) - 1))
        offset += len(t)

    # tks is a list of triples (word,start,end)
    return tks


In [18]:


def extract_features(tokens, pos_tags):
    # for each token, generate list of features and add it to the result
    result = []
    for k in range(0, len(tokens)):
        tokenFeatures = []
        t = tokens[k][0]

        tokenFeatures.append("form=" + t)
        tokenFeatures.append("formlower=" + t.lower())
        tokenFeatures.append("suf3=" + t[-3:])
        tokenFeatures.append("suf4=" + t[-4:])
        tokenFeatures.append("size=" + str(len(t)))
        if len(t) <=3:
            tokenFeatures.append("isSmall")
        if t.isupper():
            tokenFeatures.append("isUpper")
        elif t[0].isupper():
            tokenFeatures.append("isCapital")
        if t.istitle():
            tokenFeatures.append("isTitle")
        if t.isdigit():
            tokenFeatures.append("isDigit")
        if '-' in t:
            tokenFeatures.append("hasDash")

        if k > 0:
            tPrev = tokens[k - 1][0]
            tokenFeatures.append("formPrev=" + tPrev)
            tokenFeatures.append("formlowerPrev=" + tPrev.lower())
            tokenFeatures.append("suf3Prev=" + tPrev[-3:])
            tokenFeatures.append("suf4Prev=" + tPrev[-4:])
            tokenFeatures.append("postagPrev=" + pos_tags[k - 1][1])
            if (tPrev.isupper()): tokenFeatures.append("isUpperPrev")
            if (tPrev.istitle()): tokenFeatures.append("isTitlePrev")
            if (tPrev.isdigit()): tokenFeatures.append("isDigitPrev")
        else:
            tokenFeatures.append("BoS")


        if k < len(tokens) - 1:
            tNext = tokens[k + 1][0]
            tokenFeatures.append("formNext=" + tNext)
            tokenFeatures.append("formlowerNext=" + tNext.lower())
            tokenFeatures.append("suf3Next=" + tNext[-3:])
            tokenFeatures.append("suf4Next=" + tNext[-4:])
            tokenFeatures.append("postagNext=" + pos_tags[k + 1][1])
            if (tNext.isupper()): tokenFeatures.append("isUpperNext")
            if (tNext.istitle()): tokenFeatures.append("isTitleNext")
            if (tNext.isdigit()): tokenFeatures.append("isDigitNext")
        else:
            tokenFeatures.append("EoS")
            
        result.append(tokenFeatures)

    return result

data = []
for i, stext in enumerate(sentences):

    # Obtain the list of tokens in the document
    tokens = tokenize(stext)

    # Perform POS tagging
    pos_tags = pos_tag([x for (x, _, _) in tokens])

    # Take the word, POS tag, and its label
    features = extract_features(tokens, pos_tags)
    data.append(features)

In [52]:

import sys
from os import listdir

from xml.dom.minidom import parse
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import pandas as pd


In [93]:
def get_tag(token, spans):
    (form, start, end) = token
    for (spanS, spanE, spanT) in spans:
        if start == spanS and end <= spanE:
            return "B-" + spanT
        elif start >= spanS and end <= spanE:
            return "I-" + spanT

    return "O"
with open('features.train', 'w') as file:
    for f in listdir(training_data):

        # parse XML file, obtaining a DOM tree
        tree = parse(training_data + "/" + f)

        # process each sentence in the file
        sentences = tree.getElementsByTagName("sentence")
        for s in sentences:
            sid = s.attributes["id"].value  # get sentence id
            spans = []
            stext = s.attributes["text"].value  # get sentence text
            entities = s.getElementsByTagName("entity")
            for e in entities:
                # for discontinuous entities, we only get the first span
                # (will not work, but there are few of them)
                (start, end) = e.attributes["charOffset"].value.split(";")[0].split("-")
                typ = e.attributes["type"].value
                spans.append((int(start), int(end), typ))

            # convert the sentence to a list of tokens
            tokens = tokenize(stext)
            pos_tags = pos_tag([x for (x, _, _) in tokens])
            # extract sentence features
            features = extract_features(tokens, pos_tags)

            # print features in format expected by crfsuite trainer
            for i in range(0, len(tokens)):
                # see if the token is part of an entity
                tagg = get_tag(tokens[i], spans)
                joint_f = "\t".join(features[i])
                line = str(sid) + '\t'+ str(tokens[i][0]) + '\t'+ str(tokens[i][1]) + '\t'+ str(tokens[i][2]) + '\t'+ tagg + '\t'+ joint_f 
                file.write(line)

                my_new_features.append( "\t".join(features[i]))
            # blank line to separate sentences
#             print()
            file.write('\n')


In [61]:
my_new_features[0]

'form=No\tformlower=no\tsuf3=No\tsuf4=No\tsize=2\tisSmall\tisCapital\tisTitle\tBoS\tformNext=formal\tformlowerNext=formal\tsuf3Next=mal\tsuf4Next=rmal\tpostagNext=JJ'

In [81]:
train_f = []
for f in my_new_features:
    train_f.append(f.split('\t'))
train_f[0]

['form=No',
 'formlower=no',
 'suf3=No',
 'suf4=No',
 'size=2',
 'isSmall',
 'isCapital',
 'isTitle',
 'BoS',
 'formNext=formal',
 'formlowerNext=formal',
 'suf3Next=mal',
 'suf4Next=rmal',
 'postagNext=JJ']

In [80]:
def instances(fi):
    xseq = []
    yseq = y

    for i, line in enumerate(fi):
        if not line:
            # An empty line means the end of a sentence.
            # Return accumulated sequences, and reinitialize.
            yield xseq, yseq
            xseq = []
            yseq = []
            continue

        # Split the line with TAB characters.
        fields = line.split('\t')

        # Append the item features to the item sequence.
        # fields are:  0=sid, 1=form, 2=span_start, 3=span_end, 4=tag, 5...N = features
        item = fields
        xseq.append(fields)
    return xseq, yseq
        # Append the label to the label sequence.
#         yseq.append(y[i])

In [82]:
for xseq, yseq in zip(train_f,y):
    print(xseq)
    trainer.append(xseq, yseq, 0)

['form=No', 'formlower=no', 'suf3=No', 'suf4=No', 'size=2', 'isSmall', 'isCapital', 'isTitle', 'BoS', 'formNext=formal', 'formlowerNext=formal', 'suf3Next=mal', 'suf4Next=rmal', 'postagNext=JJ']


ValueError: The numbers of items and labels differ: |x| = 14, |y| = 4

In [79]:
trainer.select('l2sgd', 'crf1d')
trainer.set('c2', 0.1)

trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 0
Seconds required: 0.006

Stochastic Gradient Descent (SGD)
c2: 0.100000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 0
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: -nan
Trial #1 (eta = 0.100000): -nan (worse)
Trial #2 (eta = 0.050000): -nan (worse)
Trial #3 (eta = 0.025000): -nan (worse)
Trial #4 (eta = 0.012500): -nan (worse)
Trial #5 (eta = 0.006250): -nan (worse)
Trial #6 (eta = 0.003125): -nan (worse)
Trial #7 (eta = 0.001563): -nan (worse)
Trial #8 (eta = 0.000781): -nan (worse)
Trial #9 (eta = 0.000391): -nan (worse)
Trial #10 (eta = 0.000195): -nan (worse)
Trial #11 (eta = 0.000098): -nan (worse)
Trial #12 (eta = 0.000049): -nan (worse)
Trial #13 (eta = 0.0

In [27]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)

In [37]:
y

array(['none', 'none', 'none', ..., 'effect', 'effect', 'mechanism'],
      dtype=object)

In [36]:
len(X_train), len(y_train)

(22232, 22232)

In [41]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(y)
new_y = le.transform(y) 

In [49]:
np.array(X_train[0]).flatten()

array([list(['form=Hormonal', 'formlower=hormonal', 'suf3=nal', 'suf4=onal', 'size=8', 'isCapital', 'isTitle', 'BoS', 'formNext=Contraceptives', 'formlowerNext=contraceptives', 'suf3Next=ves', 'suf4Next=ives', 'postagNext=NNS', 'isTitleNext']),
       list(['form=Contraceptives', 'formlower=contraceptives', 'suf3=ves', 'suf4=ives', 'size=14', 'isCapital', 'isTitle', 'formPrev=Hormonal', 'formlowerPrev=hormonal', 'suf3Prev=nal', 'suf4Prev=onal', 'postagPrev=NNP', 'isTitlePrev', 'formNext=,', 'formlowerNext=,', 'suf3Next=,', 'suf4Next=,', 'postagNext=,']),
       list(['form=,', 'formlower=,', 'suf3=,', 'suf4=,', 'size=1', 'isSmall', 'formPrev=Contraceptives', 'formlowerPrev=contraceptives', 'suf3Prev=ves', 'suf4Prev=ives', 'postagPrev=NNS', 'isTitlePrev', 'formNext=Including', 'formlowerNext=including', 'suf3Next=ing', 'suf4Next=ding', 'postagNext=VBG', 'isTitleNext']),
       list(['form=Including', 'formlower=including', 'suf3=ing', 'suf4=ding', 'size=9', 'isCapital', 'isTitle', 'form

In [43]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)



In [44]:
crf.fit(X_train, new_y)

TypeError: 'numpy.int64' object is not iterable

In [50]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(X_train,y_train):
    trainer.append(np.array(xseq).flatten(), yseq, 0)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})


trainer.train('crf.model')

tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

ValueError: The numbers of items and labels differ: |x| = 46, |y| = 4